In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm.auto import tqdm
import evaluate

device = "cuda:0" if torch.cuda.is_available() else "cpu"

/home/tarit/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-27 18:00:15.011719: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-27 18:00:15.011795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-27 18:00:15.013316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 18:00:15.021397: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optim

# Load Model
Select the variants of a model to load (Standard model/LoRA model)

In [2]:
# Load Original Model / Full finetuned model
# model_repo = "model/nllb-600m-scb-3epochs"
model_repo = "wtarit/nllb-600M-th-en"
 
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_repo)

In [2]:
# Load LoRA model
from peft import PeftModel

model_repo = "facebook/nllb-200-distilled-600M"
lora_repo = "model/epochs9.0"
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo).to(device)
model = PeftModel.from_pretrained(model, lora_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo, src_lang="tha_Thai", tgt_lang="eng_Latn")

# Load test data

In [3]:
th_text = []
en_text = []

with open("../data/iwslt_2015/tst2010-2013_th-en.en", "r") as f:
    en_text = [line.strip() for line in f]
with open("../data/iwslt_2015/tst2010-2013_th-en.th", "r") as f:
    th_text = [line.strip() for line in f]

print(f"th_text: {len(th_text)}, en_text: {len(en_text)}")

th_text: 4242, en_text: 4242


In [4]:
predictions = []
batch_size = 128
for i in tqdm(range(0, len(th_text), batch_size)):
    batch = (th_text[i:i+batch_size])

    inputs = tokenizer(batch, return_tensors="pt", padding=True).to(device)

    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"), max_length=64)
    predictions += tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

100%|██████████| 34/34 [02:18<00:00,  4.09s/it]


# Calculate BLEU Score

In [5]:
metric = evaluate.load("sacrebleu")
en_ref = [[line] for line in en_text]
metric.compute(predictions=predictions, references=en_ref)

{'score': 27.376421652439596,
 'counts': [46425, 25162, 15101, 9315],
 'totals': [77942, 73700, 69458, 65223],
 'precisions': [59.56352159297939,
  34.141112618724556,
  21.741196118517664,
  14.281771767627983],
 'bp': 0.9711711922766634,
 'sys_len': 77942,
 'ref_len': 80222}